In [ ]:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil
import glob

%matplotlib inline 

In [ ]:
def conv(val):
    if val == np.nan:
        return 0
    return val

In [ ]:
df = pd.read_csv("../../diaries_transactions_all.csv", low_memory=False);


### Poverty file

In [ ]:
pov = pd.read_csv("../../pov_hh_over_trx_yr_mo_kes_excRR.csv", low_memory=False);
pov.head()


In [ ]:
pov.shape

In [ ]:
pov.describe()

### Here we check poverty info with housing conditions

In [ ]:
hou = pd.read_csv('../../housing_conditions_ownership.csv')

In [ ]:
hou.head()

In [ ]:
ids_pov = set(pov["hh_ids"].unique())
len(ids_pov)

In [ ]:
ids_hou = set(hou["hh_ids"].unique())
len(ids_hou)

In [ ]:
if (ids_pov & ids_hou):
    print(len(ids_pov & ids_hou))
else:
    print("no matches")

In [ ]:
for ids in ids_pov:
    if ids in ids_hou:
        continue
    else: print(ids)

### Check matches for the remittances as well 


In [ ]:
rem= pd.read_csv("../../rem_ind_over_trx_yr_mo_kes.csv")
rem.head()


In [ ]:
rem["m_ids_owner"].describe()

In [ ]:
rem["hh_ids"].describe()

In [ ]:
ids_rem = rem["hh_ids"].unique()


In [ ]:
for ids in ids_rem:
    if ids in ids_pov:
        continue
    else: print(ids)

In [ ]:
ids_rem = set(rem["m_ids_owner"].unique())
ids_rem.remove("HH")
rem["m_ids_owner"].value_counts()

In [ ]:
(rem["m_ids_owner"].value_counts()>1).value_counts()

### Here we are interested in hh size, monthly consumption and income 



  WE can assess whether family has enough money to cover their needs or they rely on help of relatives and friends most of the time. If monthly left is negative, then family is in debt and they are unable to take any other financil burden as they will not be able to pay it off. 

In [ ]:
pov['monthly_left'] = pov.apply(lambda row: row.inc_hh_medianpti - row.con_hh_medpti, axis = 1) 

In [ ]:
pov[['hh_ids', 'inc_hh_medianpti', 'dem_hh_size', 'con_hh_medpti', 'monthly_left']]


### Here is another file with RR included. Not a big sifference, so I assume we can exclude RR and see if the famili is not in debt without RR

In [ ]:
pov2 = pd.read_csv("../../pov_hh_over_trx_yr_mo_kes_incRR.csv", low_memory=False);
pov2.head()

In [ ]:
pov2['monthly_left'] = pov2.apply(lambda row: row.inc_hh_medianpti - row.con_hh_medpti, axis = 1) 
pov2[['hh_ids', 'inc_hh_medianpti', 'dem_hh_size', 'con_hh_medpti', 'monthly_left']]


In [ ]:
pov['monthly_left'].equals(pov2['monthly_left'])

In [ ]:
hou.dtypes

In [ ]:
hou.columns

In [ ]:
obj_hou =hou.select_dtypes(include=['all']).copy()
obj_hou.columns

In [ ]:
cleanup_nums = {"hh_inf_wall_material":     {"other": 1, "Mud/wood": 2, "Brick/block": 3, "Wood only": 4, "Corrugated Iron Sheet": 5, "Mud/cement": 6, "Stone":7 },
                "hh_inf_roof_material": {"Other": 1, "Grass, thatch, makuti": 2, "Corrugated iron, mabati": 3, "Asbestos sheet": 4,
                                  "Concrete": 5, "Tiles": 6 }, 
               "hh_inf_floor_material": {"Earth/Cowdung": 1, "Wood": 4, "Cement/Concrete/Slab":5, "Tiles": 6 },
                "hh_inf_cookfuel_source": {"Collected firewood": 1, "Charcoal": 2, "Purchased firewood": 3, "Paraffin": 4, "Gas/LPG": 5, "Eats at establishment": 6, "Biogas": 7},
              "hh_inf_lighting_source":{"Paraffin": 4, "Purchased firewood": 3,"Collected firewood": 1, "Electricity": 6, "Solar": 7, "Candles": 2, "Dry cell": 5}, 
            "hh_inf_water_source":{"Piped somewhere else (outside, neighbor)": 1, "Protected well": 2, "Protected spring": 3, "Water vendor": 4, "Unprotected well": 5, "Stream/River": 6, "Piped into dwelling": 7, "Borehole": 8, "Unprotected spring": 9, "Dam": 10, "Rain/harvested": 11},
              "hh_inf_toilet_type": {"Pit latrine, covered": 1, 
"Pit latrine, uncovered": 2, "Main sewer": 3, "Bush": 4, "Pit latrine, VIP": 5, "Septic tank": 6, "Cess pool": 7}, 
              "hh_inf_toilet_shared": {"Shared": 0, "Private": 1}, 
 'hh_inf_housing_type': {"Other temporary or semi-permanent structure": 1, "House/bungalow": 2, "Flat": 3, "Shack": 4, 
                         "Swahili type house": 5, "Maisonette/townhouse":6},               
               'hh_inf_electricity_access': {"No access": 0,"Yes, from neighbor or unofficial soure": 1, "Yes, own electricity": 2},
               'hh_inf_interiorcond': {"No major problems": 3, "Needs substantial painting, refilling, repair": 2, 
                                       "Some peeling paint, cracks in walls, rusted iron sheets": 1, "Dilapidated": 0}, 
'hh_asset_internet_access':{"Does not access internet": 0, "Other's mobile phone(s)": 1, "Cyber cafe": 2, 
                            "Own mobile phone": 3, "Own house connection through cable, dialup, mobile modem": 5,
                            "Office/workplace": 4}, 
                'hh_asset_iron_any': {"No": 0, "Yes": 1}, 
                'hh_env_settingdesc': {"Rural": 0, "Urban": 1}, 
                  
                'hh_env_desc': {"02=Single family detached houses": 8, "03=Single family attached houses": 6, 
                 "07=Open space, park, forest, farm, ranch": 7, "04=Mult- unit residential buildings (flats)": 5, 
"05=Commercial or institutional property": 4, "06=Busy road or highway": 2, "multi unit mudwalled rental houses": 3, 
"crowded houses": 1, "Anumber of rental houses with different tpye of people living in": 1, "multi unit mudwaled rental houses": 3,       
"Multi unit mudwalled rental plots": 3, "multi rental mud walled houses": 3,             
"Residentials and a busy highway": 2, "rental mud walled houses butfar apart": 3,
"muilti unit mud walled rental houses": 3, "Busy road and tea farm": 2, "multi unit mud walled rental houses": 3}, 
            'hh_env_dilapbuildings':{ "01=A few of them": 2, "98=None": 2, "02=Most of them": 1, "03=There are no buildings within 100meters": 0},
     'hh_env_trash': {"01=Minor accumulation": 0, "02=Major accumulation ": 0, "there": 1, "bushy background":2, "98=None":3}, 
                'hh_ownership_docu': {"No written agreement": 0, "Yes, a written agreement, but not a title deed": 1, "Yes, a title deed": 2}, 
               "hh_othershamba3_docu": {"01=No written agreement": 0, "03=Yes, a title deed": 1}, 
                
               'hh_otherprop_any': {"Yes": 1, "No": 0},
                'hh_otherprop1_docu': {"01=No written agreement": 0, "can have access to it": 0,
                                       "Can have access to it whenever there is need.": 0,
                                       "no documentation everyone there knows it his home": 0, 
                                       "no,but the house is ours because we are the ones who built it": 0,
                                       "He can acess it anytime he wants": 0, 
                                       "02=Yes, a written agreement, but not a title deed": 1, "its also another asset owned by the group": 1, 
                                       "03=Yes, a title deed": 2, "title deed, is with the parents": 2, "Its on a family plot": 2, 
                                       "still in his fathers home stead": 2}, 
       'hh_otherprop1_use': {"04=Agricultural property": 4, "03=Rental or commercial": 4, "03=Rental or commercial,04=Agricultural property": 4, 
                             "05=Currently unused": 0, "01=Personal use,04=Agricultural property": 3, "02=Used by friends and family": 2, "01=Personal use": 1}, 
  
                'hh_otherprop2_use': {"05=Currently unused": 0, "03=Rental or commercial": 4, "01=Personal use": 1, "04=Agricultural property": 4}, 
               'hh_otherprop2_docu': {"02=Yes, a written agreement, but not a title deed": 1, "01=No written agreement": 0, 
                                      "owned generally by the group but working towards individual ownr": 1, "03=Yes, a title deed": 2}, 
 'hh_otherprop3_docu': {"03=Yes, a title deed": 2}, 
                'hh_otherprop3_use':{"05=Currently unused": 0}, 
                'hh_othershamba1_docu':{"03=Yes, a title deed": 2, 
                 "02=Yes, a written agreement, but not a title deed": 1, "01=No written agreement": 0, 
                "inherited it from his father": 2, "have shamba number": 0, "Not collected": 0, "have an allotment lette": 1},
                
                'hh_othershamba1_use':{"04=Agricultural property": 4, "03=Rental or commercial": 4, "05=Currently unused": 0, 
                "02=Used by friends and family": 2, "01=Personal use": 1, "02=Used by friends and family,04=Agricultural property": 4}, 
                
                
                'hh_othershamba2_docu': {"02=Yes, a written agreement, but not a title deed": 1, "01=No written agreement": 0},
                'hh_othershamba2_use': {"05=Currently unused":0, "04=Agricultural property": 4}, 
       'hh_othershamba3_docu': {"01=No written agreement": 0, "03=Yes, a title deed": 2}}
                

In [ ]:
obj_hou.replace(cleanup_nums, inplace=True)
obj_hou.head()

In [ ]:
pov_hh = pd.read_csv('../../pov_hh_over_trx_yr_mo_kes_excRR.csv', dtype = {'hh_ids': 'object'})
trx = pd.read_csv('../../diaries_transactions_all.csv',dtype = {'hh_ids': 'object'}, low_memory = False)

In [ ]:
keep_ids = list(trx.m_ids_owner.unique())

In [ ]:
pov_hh.head()

In [ ]:
poverty = dfp.merge(df, how = 'left', on='hh_ids')

In [ ]:
poverty.shape

In [ ]:

pov_hou= hou_dum.merge(poverty, how = 'left', on = 'hh_ids')

In [ ]:
pov_hou.shape

In [ ]:
pov_hou_rem = dfr.merge(pov_hou, how = 'left', on = 'hh_ids')

In [ ]:
pov_hou_rem.shape

In [ ]:
pov_hou_rem.to_csv('pov_hou_rem_merge.csv')